In [1]:
# Mount google drive
import os
try:
    import google.colab
    IN_COLAB = True
except:
    IN_COLAB = False
if IN_COLAB:
    from google.colab import drive
    drive.mount('/content/drive',force_remount=True)
    base_folder = '/content/drive/My Drive/unibo/NLP_project/BarneyBot'
    os.system("pip install datasets")
    os.system("pip install transformers")
    os.system("pip install rouge_score")
    os.system("pip install -U sentence-transformers")
else:
    base_folder = os.getcwd()
    
in_folder = os.path.join(base_folder, "in")
if not os.path.exists(in_folder):
    os.makedirs(in_folder)
out_folder = os.path.join(base_folder, "out")
if not os.path.exists(out_folder):
    os.makedirs(out_folder)

In [2]:
import torch
import torchvision
import torchvision.transforms as transforms
import pandas as pd
import numpy as np
from tqdm import tqdm
from matplotlib import pyplot as plt
%matplotlib inline
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

In [3]:
character_dict = {
    'Barney':{
        'classifier_name': 'barney_classifier',
        'series_df_filename': 'HIMYM_preprocessed.csv',
        'classifier_df': 'barney_classifier.csv',
        'encoded_lines_filename': 'barney_encoded_lines.npy'
    },
    'Sheldon':{
        'classifier_name': 'sheldon_classifier'
    },
    'Harry':{
        'classifier_name': 'harry_classifier'
    },
    'Fry':{
        'classifier_name': 'fry_classifier'
    }
             }

In [4]:
batch_size = 16
epochs = 100
lr = 1e-6
regularizer_weight_r = 1e-4
regularizer_weight_s = 1e-3
dropout_rate = 0.2
train_size = 0.85
test_size = 0.10

from_saved_embeddings = True

character = 'Barney'

In [5]:
character_folder = os.path.join(base_folder, "Datasets", "Characters", character)

model_path = os.path.join(character_folder, character_dict[character]['classifier_name'])

# Dataset

In [6]:
series_df = pd.read_csv(os.path.join(character_folder, character_dict[character]['series_df_filename']))

In [7]:
series_df[series_df['character']==character]

,episode,line,character
14,01x01,"hey, so you know how I've always had a thing f...",Barney
16,01x01,"Okay, meet me at the bar in fifteen minutes, a...",Barney
18,01x01,Where's your suit!? Just once when I say suit ...,Barney
20,01x01,It was a blazer!,Barney
22,01x01,I see what this is about. Have you forgotten w...,Barney
...,...,...,...
31210,08x24,I'm probably saying some political stuff right...,Barney
31214,08x24,Whoa. Is there going to be a fight?,Barney
31449,09x10,"Karate Kid bad boy Billy Zabka, a shifty-eyed ...",Barney
31557,09x15,me or you?,Barney


In [8]:
series_df['character'] = series_df['character'].apply(lambda x: 1 if x==character else 0)

In [9]:
series_df[series_df['character']==1]

,episode,line,character
14,01x01,"hey, so you know how I've always had a thing f...",1
16,01x01,"Okay, meet me at the bar in fifteen minutes, a...",1
18,01x01,Where's your suit!? Just once when I say suit ...,1
20,01x01,It was a blazer!,1
22,01x01,I see what this is about. Have you forgotten w...,1
...,...,...,...
31210,08x24,I'm probably saying some political stuff right...,1
31214,08x24,Whoa. Is there going to be a fight?,1
31449,09x10,"Karate Kid bad boy Billy Zabka, a shifty-eyed ...",1
31557,09x15,me or you?,1


In [10]:
series_df = series_df.drop(columns=['episode'])

In [11]:
classes = (0, 1)

# Model

## Sentence Transformer

In [12]:
from sentence_transformers import SentenceTransformer

if not from_saved_embeddings:
    sentence_transformer = SentenceTransformer("sentence-transformers/paraphrase-multilingual-mpnet-base-v2")

In [13]:
if not from_saved_embeddings:
    sentence_transformer

## Sentence Encoding

In [14]:
if not from_saved_embeddings:

    series_df['encoded_line'] = [sentence_transformer.encode(line) for line in tqdm(series_df['line'])]

    # save sentences dataset
    series_df[['line', 'character']].to_csv(
        os.path.join(character_folder, character_dict[character]['classifier_df']), 
        index = False
    )

    np.save(
        os.path.join(character_folder, character_dict[character]['encoded_lines_filename']),
        series_df['encoded_line'].to_numpy()
    )

In [15]:
from numba import cuda

if not from_saved_embeddings: 
    # free gpu memory
    cuda.select_device(0)
    cuda.close()

In [16]:
# read sentences dataaset
series_df = pd.read_csv(
    os.path.join(character_folder, character_dict[character]['classifier_df']),
    dtype={'line': str,
           'character': int
          }
)

series_df['encoded_line'] = np.load(
    os.path.join(character_folder, character_dict[character]['encoded_lines_filename']), 
    allow_pickle=True
)

In [17]:
series_df

,line,character,encoded_line
0,"Kids, I'm going to tell you an incredible stor...",0,"[-0.0060349987, 0.3398651, -0.013500607, 0.051..."
1,Are we being punished for something?,0,"[0.24517804, 0.06231432, -0.015702646, 0.03040..."
2,No,0,"[0.010471302, 0.074272856, -0.015337698, 0.099..."
3,"Yeah, is this going to take a while?",0,"[0.25085106, 0.3401538, -0.0066933706, -0.1009..."
4,"Yes. Twenty-five years ago, before I was dad,...",0,"[0.07280163, 0.24518006, -0.009924358, -0.1859..."
...,...,...,...
31771,Aunt Lily wasn't wrong.,0,"[0.008160515, 0.09477018, -0.01352677, 0.13088..."
31772,"00 a.m. Christmas morning, every sleepy Sunday...",0,"[-0.17137574, -0.0025946363, -0.008149227, 0.0..."
31773,T.M.,0,"[-0.025203055, -0.27510342, -0.010736528, 0.03..."
31774,T.M.,0,"[-0.025203055, -0.27510342, -0.010736528, 0.03..."


In [18]:
def get_triplet_df(series_df, n_shuffles=1):
    
    # separate character from others
    series_df_1 = series_df[series_df['character']==1].copy()
    series_df_0 = series_df[series_df['character']==0].copy()
    
    df_rows = {'character':[], 'encoded_lines':[]}
    
    for _ in range(n_shuffles):
        # shuffle dataset
        series_df_1 = series_df_1.sample(frac=1).reset_index(drop=True)
        series_df_0 = series_df_0.sample(n=len(series_df_1)).reset_index(drop=True)
        
        for i in tqdm(range(2,len(series_df_1))):
            # character
            lines = list(series_df_1['encoded_line'][i-2:i+1])
            lines = np.concatenate(lines)
            df_rows['character'].append(1)
            df_rows['encoded_lines'].append(lines)

            # other
            lines = list(series_df_0['encoded_line'][i-2:i+1])
            lines = np.concatenate(lines)
            df_rows['character'].append(0)
            df_rows['encoded_lines'].append(lines)

    df = pd.DataFrame(data=df_rows)
    
    return df.sample(frac=1).reset_index(drop=True)

In [19]:
shuffled_df = get_triplet_df(series_df, n_shuffles=3)
shuffled_df

100%|███████████████████████████████████████████████████████████████████████████| 5140/5140 [00:00<00:00, 13698.49it/s]


,character,encoded_lines
0,0,"[-0.048949428, -0.06472829, -0.013599568, 0.02..."
1,0,"[-0.07747611, 0.02710488, -0.015132442, 0.0842..."
2,1,"[-0.0124718845, 0.179125, -0.007953476, 0.0217..."
3,0,"[0.061427712, -0.01357585, -0.013588452, 0.016..."
4,0,"[-0.07993902, -0.14875337, -0.007521808, 0.158..."
...,...,...
30835,0,"[0.057312254, 0.17898403, -0.010028928, -0.037..."
30836,0,"[-0.006781244, 0.24637458, -0.008550231, 0.018..."
30837,0,"[0.093437806, -0.03416571, -0.008995633, 0.011..."
30838,0,"[0.021078348, 0.34955186, -0.012320836, -0.192..."


## Create Classification Dataset

In [20]:
tot_len = len(shuffled_df)
train_len = int(tot_len*train_size)
test_len = int(tot_len*test_size)
val_len = tot_len - train_len - test_len

print(tot_len, train_len, test_len, val_len)

30840 26214 3084 1542


In [21]:
X_train = np.array([[float(e) for e in s] for s in shuffled_df['encoded_lines'][:train_len]])
y_train = np.array([c for c in shuffled_df['character'][:train_len]])

X_test = np.array([[float(e) for e in s] for s in shuffled_df['encoded_lines'][:test_len]])
y_test = np.array([c for c in shuffled_df['character'][:test_len]])

X_val = np.array([[float(e) for e in s] for s in shuffled_df['encoded_lines'][:val_len]])
y_val = np.array([c for c in shuffled_df['character'][:val_len]])

In [22]:
# compute some statistics
train_percentage_1 = len(y_train[y_train==1])/len(y_train)
train_percentage_0 = len(y_train[y_train==0])/len(y_train)

val_percentage_1 = len(y_val[y_val==1])/len(y_val)
val_percentage_0 = len(y_val[y_val==0])/len(y_val)
print('\t0 (%)\t\t1 (%)')
print('train\t{:.2f}\t\t{:.2f}'.format(train_percentage_0, train_percentage_1))
print('val\t{:.2f}\t\t{:.2f}'.format(val_percentage_0, val_percentage_1))

	0 (%)		1 (%)
train	0.50		0.50
val	0.52		0.48


## Classification Model

In [23]:
# Import keras/tensorflow libraries
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import callbacks
from tensorflow.keras import regularizers

In [24]:
# create model
def create_model():
    inputs = keras.Input(shape=(len(X_train[0],)))
    # x = layers.Attention(causal=True)([inputs, inputs])
    x = layers.Dense(
        1024,
        activation='relu',
        # kernel_regularizer=regularizers.l2(regularizer_weight),
        # bias_regularizer=regularizers.l2(regularizer_weight)
    )(inputs)
    x = layers.BatchNormalization()(x)
    x = layers.Dense(
        512, 
        activation='relu',
        # kernel_regularizer=regularizers.l2(regularizer_weight),
        # bias_regularizer=regularizers.l2(regularizer_weight)
    )(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dense(
        256, 
        activation='relu',
        # kernel_regularizer=regularizers.l2(regularizer_weight),
        # bias_regularizer=regularizers.l2(regularizer_weight)
    )(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dense(
        128, 
        activation='relu',
        kernel_regularizer=regularizers.l2(regularizer_weight_r),
        bias_regularizer=regularizers.l2(regularizer_weight_r)
    )(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(dropout_rate)(x)
    out = layers.Dense(
        1, 
        activation='sigmoid',
        kernel_regularizer=regularizers.l2(regularizer_weight_s),
        bias_regularizer=regularizers.l2(regularizer_weight_s)
    )(x)


    classifier_model = keras.Model(inputs, out)
    classifier_model.compile(
        loss = keras.losses.BinaryCrossentropy(),
        optimizer = keras.optimizers.Adam(learning_rate = lr),
        metrics = [keras.metrics.BinaryAccuracy(), keras.metrics.Recall()]
    )
    return classifier_model

In [25]:
model_1 = create_model()

In [26]:
model_2 = create_model()

## Training

In [27]:
earlystop_callback = callbacks.EarlyStopping(
        monitor="val_binary_accuracy",
        min_delta=0,
        patience=6,
        verbose=0,
        mode="max",
        baseline=None,
        restore_best_weights=True,
    )

In [ ]:
train_history = model_1.fit(
    X_train, 
    y_train,
    validation_data = (X_val, y_val),
    epochs= epochs,
    verbose = 1, 
    callbacks=[earlystop_callback],
    batch_size = batch_size
)

Epoch 1/100
1639/1639 [==============================] - 23s 13ms/step - loss: 0.9953 - binary_accuracy: 0.5002 - recall: 0.5048 - val_loss: 0.9338 - val_binary_accuracy: 0.5162 - val_recall: 0.4764
Epoch 2/100
1639/1639 [==============================] - 22s 14ms/step - loss: 0.9511 - binary_accuracy: 0.5123 - recall: 0.5154 - val_loss: 0.8881 - val_binary_accuracy: 0.5292 - val_recall: 0.4953
Epoch 3/100
1639/1639 [==============================] - 22s 13ms/step - loss: 0.9013 - binary_accuracy: 0.5331 - recall: 0.5376 - val_loss: 0.8516 - val_binary_accuracy: 0.5480 - val_recall: 0.5074
Epoch 4/100
 753/1639 [============>.................] - ETA: 11s - loss: 0.8802 - binary_accuracy: 0.5387 - recall: 0.5431

In [ ]:
print('#'*25 + ' Model Test ' + '#'*25)
fig, ax=plt.subplots(1,1,figsize=(5,5))
y_pred = model_1.predict(X_test).round()
# Plot the confusion matrix normalizing over the true values (over the rows)
cm = confusion_matrix(y_test, y_pred) #, normalize='pred')
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=['Others', character])
disp.plot(ax=ax)
plt.show()

## Training Recall

In [ ]:
earlystop_callback_ = callbacks.EarlyStopping(
        monitor="val_recall_1",
        min_delta=0,
        patience=6,
        verbose=0,
        mode="max",
        baseline=None,
        restore_best_weights=True,
    )

In [ ]:
train_history = model_2.fit(
    X_train, 
    y_train,
    validation_data = (X_val, y_val),
    epochs= epochs,
    verbose = 1, 
    callbacks=[earlystop_callback_],
    batch_size = batch_size,
    class_weight = class_weights
)

In [ ]:
print('#'*25 + ' Model Test ' + '#'*25)
fig, ax=plt.subplots(1,1,figsize=(5,5))
y_pred = model_2.predict(X_test).round()
# Plot the confusion matrix normalizing over the true values (over the rows)
cm = confusion_matrix(y_test, y_pred) #, normalize='pred')
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=['Others', character])
disp.plot(ax=ax)
plt.show()

In [ ]:
# classifier_path = os.path.join(character_folder, character_dict[character]['classifier_name'])
# classifier_model.save(classifier_path)